In [ ]:
# Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir -p datasets
! cp -f '/content/drive/My Drive/eva6_capstone/construction_materials_dataset.zip' datasets

In [ ]:
% cd datasets
! unzip -qq construction_materials_dataset.zip

/content/datasets


In [ ]:
! ls -l 

total 6092260
drwxr-xr-x 3 root root       4096 Aug 30 09:00  aac_blocks
drwxr-xr-x 3 root root       4096 Aug 30 09:09  adhesives
drwxr-xr-x 3 root root       4096 Aug 30 09:09  ahus
drwxr-xr-x 3 root root       4096 Aug 30 09:09  aluminium_frames_for_false_ceiling
drwxr-xr-x 3 root root       4096 Aug 30 09:09  chiller
drwxr-xr-x 3 root root       4096 Aug 30 09:09  concrete_mixer_machine
drwxr-xr-x 3 root root       4096 Aug 30 09:09 'concrete_pump_(50%)'
-r-------- 1 root root 6238268712 Sep  8 06:05  construction_materials_dataset.zip
drwxr-xr-x 3 root root       4096 Aug 30 09:09  control_panel
drwxr-xr-x 3 root root       4096 Aug 30 09:09  cu_piping
drwxr-xr-x 3 root root       4096 Aug 30 09:11  distribution_transformer
drwxr-xr-x 3 root root       4096 Aug 30 09:11  dump_truck___tipper_truck
drwxr-xr-x 3 root root       4096 Aug 30 09:11  emulsion_paint
drwxr-xr-x 3 root root       4096 Aug 30 09:11  enamel_paint
drwxr-xr-x 3 root root       4096 Aug 30 09:11  fine_aggregate


In [ ]:
% cd ..

/content


In [ ]:
! pip install git+https://github.com/cocodataset/panopticapi.git

  Cloning https://github.com/cocodataset/panopticapi.git to /tmp/pip-req-build-wg0rjipf
  Running command git clone -q https://github.com/cocodataset/panopticapi.git /tmp/pip-req-build-wg0rjipf
  Created wheel for panopticapi: filename=panopticapi-0.1-py3-none-any.whl size=8306 sha256=b49696664358e1ff71a86b0babebc0723a3c16e6a50f5f7c127e4781d113c73e
  Stored in directory: /tmp/pip-ephem-wheel-cache-um3890ux/wheels/ad/89/b8/b66cce9246af3d71d65d72c85ab993fd28e7578e1b0ed197f1
Successfully built panopticapi


In [ ]:
! cp -f   '/content/drive/My Drive/eva6_capstone/dataset_labels.txt' .

In [ ]:
from PIL import Image
import requests
import io
import math
import json
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
import pandas as pd

import torch
from torch import nn
from torchvision.models import resnet50
import torchvision.transforms as T
import numpy as np
from skimage.draw import polygon
torch.set_grad_enabled(False);

import panopticapi
from panopticapi.utils import id2rgb, rgb2id
from pycocotools import mask
import traceback

def get_segment_map(poly_segment, img_shape):
    X = [int(poly_segment[i])  for i in range(len(poly_segment)) if i%2 ==0 ]
    Y = [int(poly_segment[i])  for i in range(len(poly_segment)) if i%2 ==1 ]
    img1 = np.zeros(img_shape, dtype=np.uint8)
    rr1, cc1 = polygon(X, Y)
    img1[rr1, cc1] = 1
    img1_array = np.asfortranarray(img1)
    encoded_img1 = mask.encode(img1_array)
    return encoded_img1

# Create mappings between id and label

lines = []
with open('dataset_labels.txt') as f:
    lines = f.readlines()

id2label= {}
label2id = {}
for line in lines:
    fields = line.split('.')
    id = int(fields[0])
    label = fields[1][1:-1]
    id2label[id] = label
    label2id[label]=id
    
print("id2label:")
print(id2label)
print("label2id:")
print(label2id)

# These are the COCO classes
CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]

# Detectron2 uses a different numbering scheme, we build a conversion table
coco2d2 = {}
count = 0
for i, c in enumerate(CLASSES):
    if c != "N/A":
        coco2d2[i] = count
        count+=1

# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


id2label:
{0: 'misc_stuff', 1: 'aac_blocks', 2: 'adhesives', 3: 'ahus', 4: 'aluminium_frames_for_false_ceiling', 5: 'chiller', 6: 'concrete_mixer_machine', 7: 'concrete_pump_(50%)', 8: 'control_panel', 9: 'cu_piping', 10: 'distribution_transformer', 11: 'dump_truck___tipper_truck', 12: 'emulsion_paint', 13: 'enamel_paint', 14: 'fine_aggregate', 15: 'fire_buckets', 16: 'fire_extinguishers', 17: 'glass_wool', 18: 'grader', 19: 'hoist', 20: 'hollow_concrete_blocks', 21: 'hot_mix_plant', 22: 'hydra_crane', 23: 'interlocked_switched_socket', 24: 'junction_box', 25: 'lime', 26: 'marble', 27: 'metal_primer', 28: 'pipe_fittings', 29: 'rcc_hume_pipes', 30: 'refrigerant_gas', 31: 'river_sand', 32: 'rmc_batching_plant', 33: 'rmu_units', 34: 'sanitary_fixtures', 35: 'skid_steer_loader_(bobcat)', 36: 'smoke_detectors', 37: 'split_units', 38: 'structural_steel_-_channel', 39: 'switch_boards_and_switches', 40: 'texture_paint', 41: 'threaded_rod', 42: 'transit_mixer', 43: 'vcb_panel', 44: 'vitrified_t

In [ ]:
model, postprocessor = torch.hub.load('facebookresearch/detr', 'detr_resnet101_panoptic', pretrained=True, return_postprocessor=True, num_classes=250)
model.eval();

Downloading: "https://github.com/facebookresearch/detr/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

Downloading: "https://dl.fbaipublicfiles.com/detr/detr-r101-panoptic-40021d53.pth" to /root/.cache/torch/hub/checkpoints/detr-r101-panoptic-40021d53.pth


  0%|          | 0.00/237M [00:00<?, ?B/s]

In [ ]:
category_names1 = [ id2label[i] for i in range(1,9)]
category_names2 = [ id2label[i] for i in range(9,17)]
category_names3 = [ id2label[i] for i in range(17,25)]
category_names4 = [ id2label[i] for i in range(25,33)]
category_names5 = [ id2label[i] for i in range(33,41)]
category_names6 = [ id2label[i] for i in range(41,49)]

In [ ]:
len(category_names1)

8

In [ ]:
category_names1

['aac_blocks',
 'adhesives',
 'ahus',
 'aluminium_frames_for_false_ceiling',
 'chiller',
 'concrete_mixer_machine',
 'concrete_pump_(50%)',
 'control_panel']

In [ ]:
file_path =''

root_file_path = 'datasets/'

new_image_id = 1
new_id = 1

In [ ]:
def generate_ground_truth_for_categories(category_names):
    global new_image_id, new_id
    for category_name in category_names:
        print("Working for category:", category_name)
        json_file_path = 'datasets/' + category_name + '/coco.json'
        id_list = []
        generated_image_id_list = []
        image_path_list = []
        width_list = []
        height_list = []
        category_list = []
        segment_map_list = []
        source_list = []
        bbox_list = []
        area_list = []
        segment_polygon_list = []
        count = 0

        with open(json_file_path, 'r') as json_obj:
            contents = json.loads(json_obj.read())
            image_info_list = contents['images']
            for image_info in image_info_list:
                count += 1
                print("Working for category:", category_name, " image count: ", count, " of total: ", len(image_info_list))
                try:
                    image_id = image_info['id']
                    #generated_image_id = category_name + '_' +  str(image_id)
                    generated_image_id = new_image_id
                    new_image_id += 1
                    file_name = image_info['file_name']
                    file_path = category_name + '/images/' +  file_name
                    img_obj = Image.open(root_file_path + file_path)
                
                    if img_obj.mode != 'RGB':
                        print("Image with category: ", category_name, " file path:", file_path, " is ", img_obj.mode, " not RGB: skipping")
                        continue
                    width = img_obj.size[0]
                    height = img_obj.size[1]
                    trans_img = transform(img_obj).unsqueeze(0)
                    out = model(trans_img)
                    # compute the scores, excluding the "no-object" class (the last one)
                    scores = out["pred_logits"].softmax(-1)[..., :-1].max(-1)[0]
                    # threshold the confidence
                    keep = scores > 0.85
                    if keep.sum() == 0:
                        print("Image with  category: ", category_name, " file path:", file_path, " no bounding box with confidence higher than 0.85 found: skipping")
                    else:    
                        result = postprocessor(out, torch.as_tensor(trans_img.shape[-2:]).unsqueeze(0))[0]
                        segments_info = result["segments_info"]
                        # Panoptic predictions are stored in a special format png
                        panoptic_seg = Image.open(io.BytesIO(result['png_string']))
                        panoptic_seg_resize = panoptic_seg.resize(img_obj.size, Image.BILINEAR)
                        final_w, final_h = panoptic_seg_resize.size
                        # We convert the png into an segment id map
                        panoptic_seg_resize = np.array(panoptic_seg_resize, dtype=np.uint8)
                        panoptic_seg_resize = torch.from_numpy(rgb2id(panoptic_seg_resize))
                        for segment in segments_info:
                            if segment['category_id'] <= 91:
                                category_id = 0
                            else:
                                category_id = 48 + segment['category_id'] - 91 
                            segment_id = segment['id']
                            bbox = (out['pred_boxes'][keep][segment_id]).detach().cpu().numpy()
                            bbox_trans = [int(bbox[0]*img_obj.size[0] - 0.5*bbox[2]*img_obj.size[0]),
                                         int(bbox[1]*img_obj.size[1] - 0.5*bbox[3]*img_obj.size[1]),
                                         int(bbox[2]*img_obj.size[0]),
                                         int(bbox[3]*img_obj.size[1])]                  
                            segment_map = mask.encode(np.asfortranarray(panoptic_seg_resize.detach().cpu())==segment_id)
                            id_list.append(new_id)
                            new_id += 1
                            generated_image_id_list.append(generated_image_id)
                            image_path_list.append(file_path)
                            width_list.append(width)
                            height_list.append(height)
                            category_list.append(category_id)
                            bbox_list.append(bbox_trans)
                            segment_map_list.append(segment_map)
                            source_list.append(2)
                            area = bbox_trans[2] * bbox_trans[3]
                            area_list.append(area)
                            segment_polygon_list.append([])
                            print("Prediction: category_name:", category_name, " generated_image_id: ", generated_image_id, " file_path: ", file_path, " width: ", width, " height: ", height, " category_id: ",category_id, "bbox: ", str(bbox_trans))

                    for anno_indx in range(len(contents['annotations'])):
                        if contents['annotations'][anno_indx]['image_id'] !=  image_id:
                            continue
                        category_id = label2id[category_name]
                        poly_segment = contents['annotations'][anno_indx]['segmentation'][0]
                        segment_map = get_segment_map(poly_segment, img_obj.size)
                        id_list.append(new_id)
                        new_id += 1
                        generated_image_id_list.append(generated_image_id)
                        image_path_list.append(file_path)
                        width_list.append(width)
                        height_list.append(height)
                        category_list.append(category_id)
                        bb = contents['annotations'][anno_indx]['bbox']
                        bb = [int(bb[k]) for k in range(len(bb))]
                        bbox_list.append(bb)
                        segment_map_list.append(segment_map)
                        source_list.append(1)
                        area_list.append(contents['annotations'][anno_indx]['area'])
                        segment_polygon_list.append(poly_segment)
                        print("Annotation:  category_name:", category_name, " generated_image_id: ", generated_image_id, " file_path: ", file_path, " width: ", width, " height: ", height, " category_id: ",category_id, "bbox: ", str(bb))
                except:
                    print("Exception encounterd for category_name:", category_name, " file_path:", file_path)
                    traceback.print_exc()
                
        data = {'id': id_list, 'image_id':generated_image_id_list, 'image_path':image_path_list, 'width':width_list, 'height':height_list,  'category':category_list, 'bbox':bbox_list, 'segment_map':segment_map_list, 'source': source_list, 'area': area_list, 'segment_polygon': segment_polygon_list }
        df = pd.DataFrame(data)
        df.to_csv('/content/drive/My Drive/eva6_capstone_new/' + category_name + '.csv', index=False)
        print("Done for category:", category_name)


In [ ]:
generate_ground_truth_for_categories(category_names1)

Working for category: aac_blocks
Working for category: aac_blocks  image count:  1  of total:  263
Image with category:  aac_blocks  file path: aac_blocks/images/img_099.png  is  RGBA  not RGB: skipping
Working for category: aac_blocks  image count:  2  of total:  263
Image with category:  aac_blocks  file path: aac_blocks/images/img_098.png  is  RGBA  not RGB: skipping
Working for category: aac_blocks  image count:  3  of total:  263
Image with category:  aac_blocks  file path: aac_blocks/images/img_097.png  is  RGBA  not RGB: skipping
Working for category: aac_blocks  image count:  4  of total:  263
Image with category:  aac_blocks  file path: aac_blocks/images/img_096.png  is  RGBA  not RGB: skipping
Working for category: aac_blocks  image count:  5  of total:  263
Image with category:  aac_blocks  file path: aac_blocks/images/img_095.png  is  RGBA  not RGB: skipping
Working for category: aac_blocks  image count:  6  of total:  263
Image with category:  aac_blocks  file path: aac_bl

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/usr/local/lib/python3.7/dist-packages/torch/nn/fun

Prediction: category_name: aac_blocks  generated_image_id:  104  file_path:  aac_blocks/images/img_260.jpg  width:  500  height:  500  category_id:  148 bbox:  [0, 0, 499, 499]
Annotation:  category_name: aac_blocks  generated_image_id:  104  file_path:  aac_blocks/images/img_260.jpg  width:  500  height:  500  category_id:  1 bbox:  [35, 12, 414, 427]
Annotation:  category_name: aac_blocks  generated_image_id:  104  file_path:  aac_blocks/images/img_260.jpg  width:  500  height:  500  category_id:  1 bbox:  [0, 1, 139, 121]
Working for category: aac_blocks  image count:  105  of total:  263
Prediction: category_name: aac_blocks  generated_image_id:  105  file_path:  aac_blocks/images/img_259.jpg  width:  500  height:  500  category_id:  144 bbox:  [0, 0, 499, 499]
Annotation:  category_name: aac_blocks  generated_image_id:  105  file_path:  aac_blocks/images/img_259.jpg  width:  500  height:  500  category_id:  1 bbox:  [6, 10, 475, 467]
Working for category: aac_blocks  image count: 

Traceback (most recent call last):
  File "<ipython-input-14-e5197fd0c209>", line 88, in generate_ground_truth_for_categories
    poly_segment = contents['annotations'][anno_indx]['segmentation'][0]
IndexError: list index out of range


Prediction: category_name: ahus  generated_image_id:  378  file_path:  ahus/images/img_006.jpg  width:  337  height:  270  category_id:  156 bbox:  [0, 0, 336, 249]
Prediction: category_name: ahus  generated_image_id:  378  file_path:  ahus/images/img_006.jpg  width:  337  height:  270  category_id:  0 bbox:  [136, 35, 9, 13]
Prediction: category_name: ahus  generated_image_id:  378  file_path:  ahus/images/img_006.jpg  width:  337  height:  270  category_id:  147 bbox:  [0, 194, 336, 75]
Prediction: category_name: ahus  generated_image_id:  378  file_path:  ahus/images/img_006.jpg  width:  337  height:  270  category_id:  87 bbox:  [51, 58, 10, 13]
Prediction: category_name: ahus  generated_image_id:  378  file_path:  ahus/images/img_006.jpg  width:  337  height:  270  category_id:  0 bbox:  [141, 194, 9, 9]
Prediction: category_name: ahus  generated_image_id:  378  file_path:  ahus/images/img_006.jpg  width:  337  height:  270  category_id:  0 bbox:  [240, 90, 11, 11]
Prediction: cat

...

Prediction: category_name: chiller  generated_image_id:  591  file_path:  chiller/images/chiller (2).jpg  width:  275  height:  188  category_id:  147 bbox:  [0, 97, 274, 90]
Exception encounterd for category_name: chiller  file_path: chiller/images/chiller (2).jpg
Working for category: chiller  image count:  46  of total:  57


Traceback (most recent call last):
  File "<ipython-input-14-e5197fd0c209>", line 89, in generate_ground_truth_for_categories
    segment_map = get_segment_map(poly_segment, img_obj.size)
  File "<ipython-input-8-d572c334dea6>", line 28, in get_segment_map
    img1[rr1, cc1] = 1
IndexError: index 622 is out of bounds for axis 0 with size 275


Prediction: category_name: chiller  generated_image_id:  592  file_path:  chiller/images/chiller (19).jpg  width:  900  height:  550  category_id:  152 bbox:  [848, 216, 29, 31]
Prediction: category_name: chiller  generated_image_id:  592  file_path:  chiller/images/chiller (19).jpg  width:  900  height:  550  category_id:  156 bbox:  [0, 27, 898, 428]
Prediction: category_name: chiller  generated_image_id:  592  file_path:  chiller/images/chiller (19).jpg  width:  900  height:  550  category_id:  0 bbox:  [225, 274, 46, 31]
Prediction: category_name: chiller  generated_image_id:  592  file_path:  chiller/images/chiller (19).jpg  width:  900  height:  550  category_id:  0 bbox:  [250, 251, 97, 279]
Prediction: category_name: chiller  generated_image_id:  592  file_path:  chiller/images/chiller (19).jpg  width:  900  height:  550  category_id:  69 bbox:  [808, 242, 75, 150]
Prediction: category_name: chiller  generated_image_id:  592  file_path:  chiller/images/chiller (19).jpg  width: 

Traceback (most recent call last):
  File "<ipython-input-14-e5197fd0c209>", line 32, in generate_ground_truth_for_categories
    img_obj = Image.open(root_file_path + file_path)
  File "/usr/local/lib/python3.7/dist-packages/PIL/Image.py", line 2843, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: 'datasets/chiller/images/chiller (1).webp'
Traceback (most recent call last):
  File "<ipython-input-14-e5197fd0c209>", line 32, in generate_ground_truth_for_categories
    img_obj = Image.open(root_file_path + file_path)
  File "/usr/local/lib/python3.7/dist-packages/PIL/Image.py", line 2843, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: 'datasets/chiller/images/chiller (1).png'
Traceback (most recent call last):
  File "<ipython-input-14-e5197fd0c209>", line 32, in generate_ground_truth_for_categories
    img_obj = Image.open(root_file_path + file_path)
  File "/usr/local/lib/pyt

Prediction: category_name: chiller  generated_image_id:  603  file_path:  chiller/images/chiller (1).jpeg  width:  2000  height:  1333  category_id:  154 bbox:  [1, 140, 1996, 777]
Prediction: category_name: chiller  generated_image_id:  603  file_path:  chiller/images/chiller (1).jpeg  width:  2000  height:  1333  category_id:  82 bbox:  [1, 716, 1997, 615]
Prediction: category_name: chiller  generated_image_id:  603  file_path:  chiller/images/chiller (1).jpeg  width:  2000  height:  1333  category_id:  112 bbox:  [1641, 682, 356, 63]
Prediction: category_name: chiller  generated_image_id:  603  file_path:  chiller/images/chiller (1).jpeg  width:  2000  height:  1333  category_id:  144 bbox:  [1, 0, 1996, 616]
Annotation:  category_name: chiller  generated_image_id:  603  file_path:  chiller/images/chiller (1).jpeg  width:  2000  height:  1333  category_id:  5 bbox:  [66, 25, 469, 368]
Annotation:  category_name: chiller  generated_image_id:  603  file_path:  chiller/images/chiller (

In [ ]:
generate_ground_truth_for_categories(category_names2)

Working for category: cu_piping
Working for category: cu_piping  image count:  1  of total:  500


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Prediction: category_name: cu_piping  generated_image_id:  975  file_path:  cu_piping/images/img_050.jpg  width:  283  height:  178  category_id:  146 bbox:  [0, 0, 282, 177]
Annotation:  category_name: cu_piping  generated_image_id:  975  file_path:  cu_piping/images/img_050.jpg  width:  283  height:  178  category_id:  9 bbox:  [138, 18, 85, 76]
Annotation:  category_name: cu_piping  generated_image_id:  975  file_path:  cu_piping/images/img_050.jpg  width:  283  height:  178  category_id:  9 bbox:  [136, 92, 18, 25]
Annotation:  category_name: cu_piping  generated_image_id:  975  file_path:  cu_piping/images/img_050.jpg  width:  283  height:  178  category_id:  9 bbox:  [1, 37, 91, 77]
Annotation:  category_name: cu_piping  generated_image_id:  975  file_path:  cu_piping/images/img_050.jpg  width:  283  height:  178  category_id:  9 bbox:  [177, 95, 93, 77]
Annotation:  category_name: cu_piping  generated_image_id:  975  file_path:  cu_piping/images/img_050.jpg  width:  283  height:

Traceback (most recent call last):
  File "<ipython-input-14-e5197fd0c209>", line 88, in generate_ground_truth_for_categories
    poly_segment = contents['annotations'][anno_indx]['segmentation'][0]
IndexError: list index out of range


Streaming output truncated to the last 5000 lines.
Prediction: category_name: distribution_transformer  generated_image_id:  1795  file_path:  distribution_transformer/images/Image_140.jpeg  width:  3024  height:  4032  category_id:  138 bbox:  [183, 0, 1607, 250]
Annotation:  category_name: distribution_transformer  generated_image_id:  1795  file_path:  distribution_transformer/images/Image_140.jpeg  width:  3024  height:  4032  category_id:  10 bbox:  [300, 1040, 2692, 2388]
Working for category: distribution_transformer  image count:  322  of total:  411
Prediction: category_name: distribution_transformer  generated_image_id:  1796  file_path:  distribution_transformer/images/Image_139.jpeg  width:  3024  height:  4032  category_id:  151 bbox:  [2, 0, 3017, 4025]
Annotation:  category_name: distribution_transformer  generated_image_id:  1796  file_path:  distribution_transformer/images/Image_139.jpeg  width:  3024  height:  4032  category_id:  10 bbox:  [20, 807, 3003, 2716]
Workin